# KNN

# Model

In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.neighbors import KNeighborsRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [3]:
knn_model = KNeighborsRegressor().fit(X_train, y_train)

In [4]:
knn_model

KNeighborsRegressor()

In [5]:
knn_model.n_neighbors # ön tanımlı komşuluk değeri 5 yani 5 komşuya bakıyor, hiperparametre

5

In [6]:
knn_model.effective_metric_

'euclidean'

# Tahmin

In [7]:
y_pred = knn_model.predict(X_test)

In [8]:
np.sqrt(mean_squared_error(y_test,y_pred))

426.6570764525201

In [9]:
# şimdi farklı k değerleri için hata (RMSE) değerinin nasıl değişeceğine bakalım

In [10]:
RMSE = [] 

for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_train) 
    rmse = np.sqrt(mean_squared_error(y_train,y_pred)) 
    RMSE.append(rmse) 
    print("k =" , k , "için RMSE değeri: ", rmse)

k = 1 için RMSE değeri:  0.0
k = 2 için RMSE değeri:  179.52761335480352
k = 3 için RMSE değeri:  205.20157172291863
k = 4 için RMSE değeri:  220.5139794876305
k = 5 için RMSE değeri:  239.6467132541376
k = 6 için RMSE değeri:  243.5904190007242
k = 7 için RMSE değeri:  258.1478781634636
k = 8 için RMSE değeri:  266.05374203349805
k = 9 için RMSE değeri:  269.73782093553376
k = 10 için RMSE değeri:  271.2798300436963


In [11]:
# sadece buna bakarak train seti üzerinden cv yapmadan karar veremeyiz, o yüzden tuninge geçiyoruz

# Model Tuning

In [12]:
from sklearn.model_selection import GridSearchCV

 Bundan sonraki bölümlerin hemen hemen tümünde GridSearchCV ile hiperparametre araması yapacağız.
 
 GridSearch bir ızgara mantığı ile bir olası parametre setinin verilip tüm olası kombinasyonların denenmesi anlamına gelir.
 
#### Örneğin 2 paramtere için bazı değerler aranmak istediğinde bu 2 parametrenin birbirleriyle çarpıştırılarak olası tüm kombinasyonların denenmesi işlemini cv yöntemiyle gerçekleştirmek için kullanılan bir metoddur.

In [13]:
# KNN'de tek paramtre olmasına rağmen (tek paramtre olmasından bağımsız olan bir konu bu) bir grid (liste) oluşturacağız.
# Bu liste içinde önceki bölümlerde benzer şekilde olacak şekilde bir arama işlemi gerçekleştirilecek.  

In [14]:
knn_params = {'n_neighbors': np.arange(1,30,1)}

In [15]:
knn = KNeighborsRegressor()

In [16]:
knn_cv_model = GridSearchCV(knn, knn_params, cv = 10)

In [17]:
knn_cv_model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])})

In [18]:
knn_cv_model.best_params_["n_neighbors"] 

8

In [19]:
# valide edilmiş ve valide edilmemiş 2 grup rmse değerleri oluşturup bunları karşılaştıralım:

In [20]:
RMSE = [] 
RMSE_CV = []
for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_train) 
    rmse = np.sqrt(mean_squared_error(y_train,y_pred)) 
    rmse_cv = np.sqrt(-1*cross_val_score(knn_model, X_train, y_train, cv=10, 
                                         scoring = "neg_mean_squared_error").mean())
    RMSE.append(rmse) 
    RMSE_CV.append(rmse_cv)
    print("k =" , k , "için RMSE değeri: ", rmse, "RMSE_CV değeri: ", rmse_cv )

k = 1 için RMSE değeri:  0.0 RMSE_CV değeri:  325.3947514706382
k = 2 için RMSE değeri:  179.52761335480352 RMSE_CV değeri:  293.24000183333817
k = 3 için RMSE değeri:  205.20157172291863 RMSE_CV değeri:  283.7486667487823
k = 4 için RMSE değeri:  220.5139794876305 RMSE_CV değeri:  286.3240222024089
k = 5 için RMSE değeri:  239.6467132541376 RMSE_CV değeri:  290.0705466132226
k = 6 için RMSE değeri:  243.5904190007242 RMSE_CV değeri:  298.1263115575851
k = 7 için RMSE değeri:  258.1478781634636 RMSE_CV değeri:  294.77070479194987
k = 8 için RMSE değeri:  266.05374203349805 RMSE_CV değeri:  291.98672028891235
k = 9 için RMSE değeri:  269.73782093553376 RMSE_CV değeri:  295.7162739573105
k = 10 için RMSE değeri:  271.2798300436963 RMSE_CV değeri:  301.31047022701154


In [21]:
# Final Modeli

Q: Hocam Knn Model Tuning yaptıktan sonra optimum k değerini 8 bulduk. Fakat valide edilmiş modelde RMSE değeri k=8 olduğunda en düşük değil. Neden optimum olarak 8'i aldık? Hatayı minimize etmesi gerekmez miydi optimum değerin?

A: best_params fonksiyonu RMSE değerine bakarak bir sırlama yapmadığı için böyle bir durumla karşılaşılıyor.  best_params fonksiyonu "mean_validation_score" değerine göre bir sıralama yapıyor. Bu nedenle RMSE değerlerine bakıldığında farklı bir sonuç çıkıyor.  Keşke bu durumdan eğitimde bahsedilseydi. Zira en düşük hataya bakıp en düşük hatalı olmayan k değerini seçmek çok mantıklı değil.

In [22]:
# Yukarıdaki soruda neden optimum değerin rmse'si en düşük olan değer yani 2 değil de 8 olduğunu anlıyorız, burada da 8 kullan

In [23]:
knn_tuned = KNeighborsRegressor(n_neighbors = knn_cv_model.best_params_["n_neighbors"]).fit(X_train, y_train)

In [24]:
# yukarda tune edilmiş modelin nesnesini oluşturduk ve train setlerimizi fit ettik

In [25]:
# final test (rmse) score hesaplama yapıyoruz son olarak

In [26]:
y_pred = knn_tuned.predict(X_test) #predictionlarımızı yaptık

In [27]:
np.sqrt(mean_squared_error(y_test, y_pred))

413.7094731463598